# 1. 転置インデックスとマッチング

（想定実行時間：それぞれ1分以内）



## 1. ディクショナリ

英語製品データについて、以下の連想配列を構築せよ。
ただし、単語はスペースで区切られているものとする（以降の問題でも同じ）。

- キー： `product_title` フィールドに出現する単語。
- 値：その単語の出現回数。ただし、全ての製品における合計とする。

また、単語の異なり数が約90万であることを確かめよ。



### 回答


In [26]:
import pandas as pd
from pandas import DataFrame
import pathlib
from typing import Dict
import time

def print_perf(func):
    def wrapper(*args, **kargs):
        start = time.perf_counter_ns()
        result = func(*args, **kargs)
        end = time.perf_counter_ns()
        print(f" {func.__name__}: {format((end - start)/1000000, '.3f')}ms {format(end - start, '.3f')}ns")
        return result
    return wrapper


# DataFrameの取り出し
@print_perf
def load_product_dataframe() -> DataFrame:
    esci_path = pathlib.Path("../esci-data/shopping_queries_dataset")
    print("Start loading products...")
    df_products = pd.read_parquet(esci_path.joinpath("shopping_queries_dataset_products.parquet"))
    print("Loaded products.")
    df_products_us = df_products[df_products["product_locale"] == "us"] 
    return df_products_us

@print_perf
def create_product_title_dictionary(products: DataFrame) -> Dict[str, int]:
    print("Start counting...")
    dict_split_default: Dict[str, int] = {}
    for title in products["product_title"]:
        for word in title.split():
            # 辞書に数を数えながら追加
            if len(word) > 0:
                if word in dict_split_default:
                    dict_split_default[word] = dict_split_default[word] + 1
                else:
                    dict_split_default[word] = 1
    return dict_split_default

df_products = load_product_dataframe()

title_dictionary = create_product_title_dictionary(df_products)

len(title_dictionary)



Start loading products...
Loaded products.
 load_product_dataframe: 5721.454ms 5721453796.000ns
Start counting...
 create_product_title_dictionary: 5815.825ms 5815825232.000ns


912923

## 2. ポスティングリスト

単語 `Information` を `product_title` フィールドに含む製品の `product_id` の配列を出力せよ。
ただし、大文字と小文字を区別するものとする（例えば `Information` と `information` は異なる単語とする。以降の問題でも同じ）。
また、`product_id` は辞書順とし、重複してはならない（以降の問題でも同じ）。

また、この配列の要素数が110であることを確かめよ。



### 回答


In [27]:
from typing import List

@print_perf
def find_products_with_word_in_title(products: DataFrame, word: str) -> List[str]:
    ids = []
    print("Start counting...")
    sorted_products = products.sort_values("product_id")
    for product in sorted_products.itertuples():
        if word in product.product_title.split():
            ids.append(product.product_id)
    return ids

# InformationのIDのリスト
ids = find_products_with_word_in_title(df_products, "Information") 

print(f"Postings list size of \"Information\" is {len(ids)}")



Start counting...
 find_products_with_word_in_title: 5603.820ms 5603819783.000ns
Postings list size of "Information" is 110


## 3. 転置インデックス

以下の連想配列を構築せよ：

- キー：`product_title` フィールドに出現する単語。
- 値：その単語を `product_title` フィールドに含む製品の `product_id` の配列。

また、この連想配列について、以下のことを確かめよ：

- エントリ数が `1.` の連想配列のそれと等しいこと。
- `Information` に紐づいた配列が `2.` の配列と等しいこと。

以降、このデータ構造を**転置インデックス**、転置インデックスの値を**ポスティングリスト**と呼ぶ。



### 回答

2ですでに作ってしまったので、数の確認だけにとどめる。

In [28]:
@print_perf
def create_product_title_inverted_index(products: DataFrame) -> Dict[str,List[str]]:
    print("Start creating inverted index...")
    inverted_index = {}
    sorted_products = products.sort_values("product_id")
    for product in sorted_products.itertuples():
        words = product.product_title.split()
        words.sort()
        prev_word = ""
        for word in words:
            if prev_word != word:
                if word in inverted_index:
                    inverted_index[word].append(product.product_id)
                else:
                    inverted_index[word] = [product.product_id]
            prev_word = word
    return inverted_index

title_inverted_index = create_product_title_inverted_index(df_products)

print(f"エントリー数が等しいかどうか？ {len(title_dictionary) == len(title_inverted_index)}")
print(f"Informationの配列が等しいかどうか？ {len(ids) == len(title_inverted_index['Information'])}")


Start creating inverted index...
 create_product_title_inverted_index: 16538.425ms 16538425266.000ns
エントリー数が等しいかどうか？ True
Informationの配列が等しいかどうか？ True


## 4. 永続化

転置インデックスをメモリからディスクに書き込んだり、ディスクからメモリに読み込んだりできるようにせよ。



### 回答


In [29]:
import pickle
from pathlib import Path

@print_perf
def dump(file: Path, inverted_index: Dict[str, List[str]]):
    with dump_file.open(mode="wb") as dwf:
        pickle.dump(obj=inverted_index, file=dwf)

@print_perf
def load(file: Path) -> Dict[str,List[str]]:
    with file.open(mode="rb") as drf:
        loaded_dict = pickle.load(file=drf)
    return loaded_dict

dump_file = pathlib.Path("./inverted_index.dump")

dump(file=dump_file, inverted_index=title_inverted_index)
loaded_dict = load(file=dump_file)

ids = list(loaded_dict["Information"])
ids.sort()
print(f"Postings list size of \"Information\" is {len(ids)}")
ids

 dump: 4175.818ms 4175818347.000ns
 load: 5170.530ms 5170529719.000ns
Postings list size of "Information" is 110


['0133741621',
 '0134296540',
 '0135217725',
 '0262031639',
 '0399159258',
 '073860836X',
 '0738610364',
 '0804717265',
 '0823085546',
 '0993294812',
 '107014598X',
 '1081837160',
 '1097276546',
 '1099719615',
 '1099741408',
 '1099998190',
 '1118674367',
 '1118890795',
 '1118988531',
 '111900294X',
 '1119101603',
 '1138366404',
 '1426215436',
 '1440574561',
 '1440854475',
 '1440875049',
 '1441317295',
 '1441317996',
 '1491911689',
 '1510750789',
 '1545261261',
 '1593995520',
 '1598849891',
 '1619549506',
 '1631592963',
 '1688868674',
 '1690155701',
 '1695096584',
 '1697447031',
 '1796683000',
 '1796688916',
 '1798422395',
 '1938377001',
 '3030418189',
 'B000RR4AU2',
 'B000XYT3IS',
 'B000XYT3J2',
 'B000XYT3JC',
 'B000XYT3JM',
 'B000Y2TU7S',
 'B000Y2TU82',
 'B0019WTCYS',
 'B002QX43ZM',
 'B00BZA22B4',
 'B00BZA22BE',
 'B00G334QXU',
 'B00OVNI8XI',
 'B00RZJ2Y2Q',
 'B00T0ROWE4',
 'B00U9AJB24',
 'B00VEYRZBI',
 'B010CKGBVE',
 'B018OERCSQ',
 'B01EHX2BH0',
 'B01H301G4I',
 'B01IL2G2XW',
 'B06XBPJV

## 5. ブーリアンAND検索

転置インデックスを用いて、2単語 `Information` と `Science` を**ともに** `product_title` フィールドに含む製品の `product_id` の配列を出力せよ。
ただし、配列が非常に長い場合にも効率的なコードにせよ（以降の問題でも同じ）。

また、この配列の要素数が3であることを確かめよ。



### 回答


In [35]:
info_list = loaded_dict["Information"]
sci_list = loaded_dict["Science"]

@print_perf
def intersection_using_set(list1: List[str], list2: List[str]) -> List[str]:
    return list(set(list1).intersection(list2))

@print_perf
def intersection(list1: List[str], list2: List[str]) -> List[str]:
    while True:
        
        break
    pass

intersection_list = intersection_using_set(list1=info_list, list2=sci_list)
print(f"Length = {len(intersection_list)}")


 intersection_using_set: 0.269ms 268551.000ns
Length = 3


## 6. ブーリアンOR検索

転置インデックスを用いて、2単語 `Information` と `Retrieval` の**少なくとも片方を** `product_title` フィールドに含む製品の `product_id` の配列を出力せよ。

また、この配列の要素数が129であることを確かめよ。



### 回答


In [36]:
ret_list = loaded_dict["Retrieval"]

@print_perf
def union_using_set(list1: List[str], list2: List[str]) -> List[str]:
    return sorted(list(set(list1).union(list2)))

union_list = union_using_set(info_list, ret_list)
print(f"Length = {len(union_list)}")
union_list

 union_using_set: 0.080ms 79908.000ns
Length = 129


['0133741621',
 '0134296540',
 '0135217725',
 '0262031639',
 '0399159258',
 '073860836X',
 '0738610364',
 '0804717265',
 '0823085546',
 '0993294812',
 '107014598X',
 '1081837160',
 '1097276546',
 '1099719615',
 '1099741408',
 '1099998190',
 '1118674367',
 '1118890795',
 '1118988531',
 '111900294X',
 '1119101603',
 '1138366404',
 '1426215436',
 '1440574561',
 '1440854475',
 '1440875049',
 '1441317295',
 '1441317996',
 '1491911689',
 '1510750789',
 '1545261261',
 '1593995520',
 '1598849891',
 '1619549506',
 '1631592963',
 '1688868674',
 '1690155701',
 '1695096584',
 '1697447031',
 '1796683000',
 '1796688916',
 '1798422395',
 '1938377001',
 '3030418189',
 'B000RR4AU2',
 'B000XYT3IS',
 'B000XYT3J2',
 'B000XYT3JC',
 'B000XYT3JM',
 'B000Y2TU7S',
 'B000Y2TU82',
 'B0019WTCYS',
 'B001TH8DIO',
 'B002QX43ZM',
 'B0030G7YSM',
 'B00BZA22B4',
 'B00BZA22BE',
 'B00G334QXU',
 'B00N4TDRCC',
 'B00OVNI8XI',
 'B00PMZ8QKY',
 'B00Q3HJRJE',
 'B00Q3HKEI2',
 'B00QLZLWNW',
 'B00RZJ2Y2Q',
 'B00T0ROWE4',
 'B00U9AJB

## 7. 条件の否定

`5.` の条件を**満たさず**、かつ、`6.` の条件を満たす製品の `product_id` の配列を出力せよ。

また、この配列の要素数が126であることを確かめよ。



### 回答


In [38]:
@print_perf
def difference_using_set(base: List[str], negate: List[str]) -> List[str]:
    result = base
    for id in negate:
        if id in base:
            result.remove(id)
    return result
        
result = difference_using_set(base=union_list, negate=intersection_list)
print(f"Length = {len(result)}")
result


 difference_using_set: 0.016ms 15820.000ns
Length = 126


['0133741621',
 '0134296540',
 '0135217725',
 '0262031639',
 '0399159258',
 '073860836X',
 '0738610364',
 '0804717265',
 '0823085546',
 '0993294812',
 '107014598X',
 '1081837160',
 '1097276546',
 '1099719615',
 '1099741408',
 '1099998190',
 '1118674367',
 '1118890795',
 '1118988531',
 '111900294X',
 '1119101603',
 '1138366404',
 '1426215436',
 '1440574561',
 '1441317295',
 '1441317996',
 '1491911689',
 '1510750789',
 '1545261261',
 '1593995520',
 '1619549506',
 '1631592963',
 '1688868674',
 '1690155701',
 '1695096584',
 '1697447031',
 '1796683000',
 '1796688916',
 '1798422395',
 '1938377001',
 '3030418189',
 'B000RR4AU2',
 'B000XYT3IS',
 'B000XYT3J2',
 'B000XYT3JC',
 'B000XYT3JM',
 'B000Y2TU7S',
 'B000Y2TU82',
 'B0019WTCYS',
 'B001TH8DIO',
 'B002QX43ZM',
 'B0030G7YSM',
 'B00BZA22B4',
 'B00BZA22BE',
 'B00G334QXU',
 'B00N4TDRCC',
 'B00OVNI8XI',
 'B00PMZ8QKY',
 'B00Q3HJRJE',
 'B00Q3HKEI2',
 'B00QLZLWNW',
 'B00RZJ2Y2Q',
 'B00T0ROWE4',
 'B00U9AJB24',
 'B00VEYRZBI',
 'B010CKGBVE',
 'B018OERC

## 8. 完全一致

`product_brand` フィールドに関する転置インデックスを構築せよ。
ただし単語に区切らず、フィールドの値をそのまま連想配列のキーとすること。



### 回答


In [40]:

@print_perf
def create_product_brand_inverted_index(products: DataFrame) -> Dict[str,List[str]]:
    print("Start creating inverted index...")
    inverted_index = {}
    sorted_products = products.sort_values("product_id")
    # product_brandのカラムでループ処理
    for product in sorted_products.itertuples():
        brand = product.product_brand
        if brand in inverted_index:
            inverted_index[brand].append(product.product_id)
        else:
            inverted_index[brand] = [product.product_id]
    return inverted_index

brand_postings = create_product_brand_inverted_index(df_products)
len(brand_postings)

Start creating inverted index...
 create_product_brand_inverted_index: 6188.001ms 6188001410.000ns


211171

## 9. 複数フィールドの横断

転置インデックスを用いて、`product_title` フィールドに単語 `Amazon` を含み、かつ、`product_brand` フィールドの値が `Amazon Basics` **ではない**製品の数が8,681であることを確かめよ。



### 回答


In [41]:
amazon_title_list = loaded_dict["Amazon"]
amazon_basics_brand_list = brand_postings["Amazon Basics"]

result = difference_using_set(base=amazon_title_list, negate=amazon_basics_brand_list)

len(result)


 difference_using_set: 395.248ms 395248085.000ns


8681

## 10. プレフィックス

`3.` で構築した転置インデックスを圧縮する。
`product_id` が固定長であり、かつ辞書順にソートされていることを利用して、ポスティングリスト上で前の `product_id` と共通のプレフィックスを省略せよ。
このとき、ポスティングリストのイテレータ（`product_id` を復元して返す）も実装せよ。



### 回答


## 11. 符号化

前問で構築した転置インデックスを更に圧縮する。
ポスティングリストとは別に全ての `product_id` の辞書順の配列を保存しておき、そのインデックスによってポスティングリストから `product_id` を参照するようにせよ。
また、イテレータも実装せよ。



### 回答


## 12. Variable Byte Encoding

前問で構築した転置インデックスを更に圧縮する。
ポスティングリスト中のインデックスが単調増加することを利用して、前のインデックスからの差を保存せよ。
このとき、差は小さな整数になると期待されるので、variable byte encodingにより圧縮せよ。
また、イテレータも実装せよ。


### 回答
